In [ ]:
! python --version

In [ ]:
! pip freeze | grep -P '(torch|numpy)'

In [ ]:
! lscpu

In [ ]:
! nvcc --version

In [ ]:
! nvidia-smi --query-gpu=name,driver_version --format=csv

In [ ]:
import torch
torch.set_printoptions(precision=11)

In [ ]:
print(torch.__config__.show())

In [ ]:
print('Cuda', torch.cuda.is_available())

In [ ]:
def set_random_seed(seed_value: int, use_cuda: bool = False):
    import torch
    import numpy as np
    import random
    np.random.seed(seed_value)  # cpu vars
    torch.manual_seed(seed_value)  # cpu  vars
    random.seed(seed_value)  # Python
    torch.use_deterministic_algorithms(True)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # gpu vars
        torch.backends.cudnn.deterministic = True  # needed
        torch.backends.cudnn.benchmark = True
    

def _run_module(layer_norm, dropout, x, use_cuda, train, seed):
    if use_cuda:
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    set_random_seed(seed, use_cuda=use_cuda)
    layers = []
    if layer_norm:
        layers.append(torch.nn.LayerNorm(normalized_shape=(x.shape[-1],), eps=1e-12, elementwise_affine=True))
    if dropout:
        layers.append(torch.nn.Dropout(0.5, inplace=False))
        
    if not layers:
        raise ValueError('set `layer_norm` and/or `dropout` to True')
    
    model = torch.nn.Sequential(*layers)
    model.to(device)
    if train:
        model.zero_grad()
        model.train()
    else:
        model.eval()
    x = x.clone().to(device)
    if train:
        out = model(x)
    else:
        with torch.no_grad():
            out = model(x)
    return out


def run_layer_norm_module(seed):
    set_random_seed(seed, use_cuda=False)
    x = torch.rand(3, 3)
    return {
        'x': x,
        'cpu_t': _run_module(layer_norm=True, dropout=False, x=x, use_cuda=False, train=True, seed=seed),
        'cpu_e': _run_module(layer_norm=True, dropout=False, x=x, use_cuda=False, train=False, seed=seed),
        'gpu_t': _run_module(layer_norm=True, dropout=False, x=x, use_cuda=True, train=True, seed=seed),
        'gpu_e': _run_module(layer_norm=True, dropout=False, x=x, use_cuda=True, train=False, seed=seed)
    }



def run_dropout_module(seed):
    set_random_seed(seed, use_cuda=False)
    x = torch.rand(4, 256, 256)
    return {
        'x': x,
        'cpu_t': _run_module(layer_norm=False, dropout=True, x=x, use_cuda=False, train=True, seed=seed),
        'gpu_t': _run_module(layer_norm=False, dropout=True, x=x, use_cuda=True, train=True, seed=seed),
    }

In [ ]:
runs = [run_dropout_module(seed=42) for i in range(5)]

for i in range(1, len(runs)):
    for k in runs[0]:
        assert torch.equal(runs[0][k], runs[i][k]), (i, k)
        
for k in runs[0]:
    if k != 'x':
        print(k, (runs[0][k].cpu() == 0.0).sum(axis=-1).tolist())
        print('=' * 100)
    

In [ ]:
runs = [run_layer_norm_module(seed=42) for i in range(5)]

for i in range(1, len(runs)):
    for k in runs[0]:
        assert torch.equal(runs[0][k], runs[i][k]), (i, k)
        
runs[0]